## Timing

In [2]:
import timeit
import time

In [2]:
def time_consuming_task(t=5):
    print(f"\tworking hard... (for {t} s)")
    time.sleep(t)

### timeit

In [3]:
numbers = [7, 6, 1, 4, 1, 8, 0, 6]
set(numbers)

{0, 1, 4, 6, 7, 8}

Solutions for Jupyter notebooks:

In [4]:
%timeit set(numbers) # Only in ipynb

190 ns ± 2.44 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [5]:
%timeit {*[7, 6, 1, 4, 1, 8, 0, 6]}

182 ns ± 2.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
timeit.timeit('{*[7, 6, 1, 4, 1, 8, 0, 6]}')

0.175687000000039

### time decorator

Measure the execution time of a function.

In [7]:
import functools


def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

In [8]:
@timer
def do_task():
    time_consuming_task(3)


do_task()

	working hard... (for 3 s)
Elapsed time: 3.0124 seconds


Only the function with the decorator is measured. No information about the time spent in subfunctions.

In [9]:
@timer
def do_tasks():
    time_consuming_task(3)
    time_consuming_task(1)
    time_consuming_task(0.5)


do_tasks()

	working hard... (for 3 s)
	working hard... (for 1 s)
	working hard... (for 0.5 s)
Elapsed time: 4.5152 seconds


Trick to add decorator to an already defined function:

In [12]:
@timer
def do_tasks_detailed():
    timed_task = timer(time_consuming_task)
    timed_task(3)
    timed_task(1)
    timed_task(0.5)


do_tasks_detailed()

	working hard... (for 3 s)
Elapsed time: 3.0131 seconds
	working hard... (for 1 s)
Elapsed time: 1.0016 seconds
	working hard... (for 0.5 s)
Elapsed time: 0.5036 seconds
Elapsed time: 4.5197 seconds


More detailed info of the function being executed:

In [13]:
import functools


def timer2(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time in {func.__name__}({args=}, {kwargs=}): {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

In [14]:
@timer2
def do_tasks_detailed():
    timed_task = timer2(time_consuming_task)
    timed_task(3)
    timed_task(1)
    timed_task(0.5)


do_tasks_detailed()

	working hard... (for 3 s)
Elapsed time in time_consuming_task(args=(3,), kwargs={}): 3.0144 seconds
	working hard... (for 1 s)
Elapsed time in time_consuming_task(args=(1,), kwargs={}): 1.0131 seconds
	working hard... (for 0.5 s)
Elapsed time in time_consuming_task(args=(0.5,), kwargs={}): 0.5142 seconds
Elapsed time in do_tasks_detailed(args=(), kwargs={}): 4.5442 seconds


# Profiling

In [10]:
import cProfile


def foo():
    for i in range(1_000):
        i**2
        if i % 10 == 0:
            time.sleep(0.0001)


# Profile 1 function call
cProfile.run('foo()')

         104 function calls in 1.507 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    1.507    1.507 530314923.py:3(foo)
        1    0.000    0.000    1.507    1.507 <string>:1(<module>)
        1    0.000    0.000    1.507    1.507 {built-in method builtins.exec}
      100    1.505    0.015    1.505    0.015 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [7]:
import pstats

res = cProfile.run('foo()', 'foo()')
print(res)

p = pstats.Stats('foo()')

print(p.sort_stats('time').print_stats(3))

None
Wed Jun 14 16:39:08 2023    foo()

         104 function calls in 1.514 seconds

   Ordered by: internal time
   List reduced from 5 to 3 due to restriction <3>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    1.511    0.015    1.511    0.015 {built-in method time.sleep}
        1    0.002    0.002    1.514    1.514 C:\Users\peter.mizsak\AppData\Local\Temp\ipykernel_14516\3324104090.py:4(foo)
        1    0.000    0.000    1.514    1.514 <string>:1(<module>)




The `cprofile` module is used to profile the execution time of a program, providing detailed information on the execution time of each function.
With the `pstats` module, the results of the profiling can be analyzed.

The `cprofile` module can be used as a context manager or directly from the command line:


### As a context manager: 
```python
with cProfile.Profile() as pr:
    #do something
    pr.print_stats()
```

### Directly from the command line:

```bash
python -m cProfile -o output_file.prof script.py
```

The results can be analyzed with the `pstats` module:

```python
import pstats
p = pstats.Stats('output_file.prof')
p.sort_stats('cumulative').print_stats(10)
```


Context manager example:

In [9]:
with cProfile.Profile() as pr:
    foo()
    pr.print_stats()

         110 function calls in 1.511 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    1.511    1.511 3324104090.py:4(foo)
        1    0.000    0.000    0.000    0.000 cProfile.py:40(print_stats)
        1    0.000    0.000    0.000    0.000 cProfile.py:50(create_stats)
        1    0.000    0.000    0.000    0.000 pstats.py:118(load_stats)
        1    0.000    0.000    0.000    0.000 pstats.py:89(__init__)
        1    0.000    0.000    0.000    0.000 pstats.py:99(init)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
      100    1.509    0.015    1.509    0.015 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


